In [47]:
# !python -V

### Preprocessing Data

In [48]:
# import libraries for data preprocessing

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error

In [49]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [50]:
import mlflow

In [51]:
# mlflow.delete_experiment(0)
#
# mlflow.start_run()
# run = mlflow.active_run()
# print("Active run_id: {}".format(run.info.run_id))
# mlflow.end_run()

In [52]:
# sqlite is used as mlflow backend and set the experiment name for tracking
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment_linear")

2024/06/18 15:08:39 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment_linear' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/drago/Documents/Ironhack/Week8/lab-ml-ops/mlruns/3', creation_time=1718716119450, experiment_id='3', last_update_time=1718716119450, lifecycle_stage='active', name='nyc-taxi-experiment_linear', tags={}>

### Train using Linear Regressor and set the experiment to mlflow manually

In [53]:
# Preprocessing Data
# function to read the data and remove outliers

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df



In [54]:
# call read and preprocessing functions for train, validation dataset
df_train = read_dataframe('Data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('Data/green_tripdata_2021-02.parquet')

In [55]:
# info of dataset
len(df_train), len(df_val)

(73908, 61921)

In [56]:
# feature engineering
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [57]:
# fit the dictvectorizer and preprocess data
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [58]:
# 1. Train a linear model (without mlflow)
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

# mean_squared_error(y_val, y_pred, squared=False)

In [59]:
import pickle

In [60]:
with open('lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [61]:
# Train another linear model (Lasso)
# lr = Lasso(0.01)
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_val)

# mean_squared_error(y_val, y_pred, squared=False)

In [62]:
# 2. Train a linear model (with mlflow)

with mlflow.start_run():

    mlflow.set_tag("developer", "Amber")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)

    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")


c:\Users\drago\Documents\Ironhack\.venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
